In [3]:
from fer import FER
import matplotlib.pyplot as plt 
from fer import Video
import os
import sys
import pandas as pd
import torch
from pydub import AudioSegment
from pydub.playback import play  
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from pydub import AudioSegment
# Put in the location of the video file that has to be processed
location_videofile = "C:/Users/Razai/one_person_Trim_5.mp4"


# Build the Face detection detector
face_detector = FER(mtcnn=True)
# Input the video for processing
input_video = Video(location_videofile)

# The Analyze() function will run analysis on every frame of the input video. 
# It will create a rectangular box around every image and show the emotion values next to that.
# Finally, the method will publish a new video that will have a box around the face of the human with live emotion values.
processing_data = input_video.analyze(face_detector, display=False)

# We will now convert the analysed information into a dataframe.
# This will help us import the data as a .CSV file to perform analysis over it later
vid_df = input_video.to_pandas(processing_data)
vid_df = input_video.get_first_face(vid_df)
vid_df = input_video.get_emotions(vid_df)

# Plotting the emotions against time in the video
#pltfig = vid_df.plot(figsize=(20, 8), fontsize=16).get_figure()

# We will now work on the dataframe to extract which emotion was prominent in the video
angry = sum(vid_df.angry)
disgust = sum(vid_df.disgust)
fear = sum(vid_df.fear)
happy = sum(vid_df.happy)
sad = sum(vid_df.sad)
surprise = sum(vid_df.surprise)
neutral = sum(vid_df.neutral)

emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotions_values = [angry, disgust, fear, happy, sad, surprise, neutral]

score_comparisons = pd.DataFrame(emotions, columns = ['Human Emotions'])
score_comparisons['Emotion Value from the Video'] = emotions_values


mp4_file = location_videofile
track = AudioSegment.from_file(location_videofile,  format= 'mp4')
wav_filename ="wav_filename.wav"
file_handle = track.export(wav_filename, format='wav')

# sound = AudioSegment.from_file(wav_filename, format='wav', frame_rate=16000)
# sound = sound.set_frame_rate(16000)
# sound.export("C:/Users/Razai/asd1.wav", format='wav')
# sound_new = AudioSegment.from_file(wav_filename, format="wav")
#new_mp3_audio= mp3_audio + 10 # increase volume of audio
#mp3_audio = AudioSegment.from_file(r"Debate_Talking5min.wav", format="wav")
#print(len(mp3_audio)/(1000*60))

mp3_audio = AudioSegment.from_file(wav_filename, format="wav")
# print("----------Before Conversion--------")
# print("Frame Rate", mp3_audio.frame_rate)
# print("Channel", mp3_audio.channels)
# print("Sample Width",mp3_audio.sample_width)
# Change Frame Rate
mp3_audio = mp3_audio.set_frame_rate(16000)
# Change Channel
mp3_audio = mp3_audio.set_channels(1)
# Change Sample Width
mp3_audio = mp3_audio.set_sample_width(2)
# Export the Audio to get the changed 
mp3_audio.export(wav_filename, format ="wav")
#print(mp3_audio.frame_rate)


# 12 Minutes audio breaks into 3 minutes 4 audio files (slicing is done by milliseconds)

counter_audio = 120
split_audio = [mp3_audio[:120*1000]]
for i in range(4):
    split_audio.append(mp3_audio[counter_audio*1000:(counter_audio+120)*1000])
    counter_audio += 120

count = 0
# # lets save it!
for count, audio_object in enumerate(split_audio):
    count += 1
    with open(f"{count}_wav_filename.wav", 'wb') as out_f:
        audio_object.export(out_f, format='wav')
        
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
collection_of_text = []
for i in range(4):

    speech, rate = librosa.load(f"{i+1}_wav_filename.wav", sr=16000)

    input_values = tokenizer(speech, return_tensors='pt').input_values
    # Store logits (non-normalized predictions)
    with torch.no_grad():
        logits = model(input_values).logits

    # Store predicted id's
    predicted_ids = torch.argmax(logits, dim=-1)
    # decode the audio to generate text
    # Passing the prediction to the tokenzer decode to get the transcription
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    # transcriptions = tokenizer.decode(predicted_ids[0])
    print(transcription)
    collection_of_text.append(transcription)

print(collection_of_text)
final_complete_speech = ""

# convert batch of text into one complete sentence
for i in collection_of_text:
    final_complete_speech += i
print(final_complete_speech)
print(angry)
print(disgust)
print(fear)
print(happy)
print(sad)
print(surprise)
print(neutral)

output_file = open('emotion_value.txt', 'w')
for emotion in emotions_values :
    output_file.write(f'{emotion}\n')
output_file.close()

with open('recognized.txt',mode ='w') as file: 
    file.write("Recognized Speech:") 
    file.write("\n") 
    file.write(final_complete_speech)

INFO:fer:30.00 fps, 162 frames, 5.40 seconds
INFO:fer:Making directories at output
INFO:fer:Deleted pre-existing output\one_person_Trim_5_output.mp4
100%|████████████████████████████████████████████████████████████████████████████| 162/162 [01:11<00:00,  2.28frames/s]
INFO:fer:Completed analysis: saved to output\one_person_Trim_5_output.mp4
INFO:fer:Starting to Zip
INFO:fer:Compressing: 30%
INFO:fer:Compressing: 61%
INFO:fer:Compressing: 92%
INFO:fer:Zip has finished
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to b

NOW I'M GOING TO GIVE YOU A TOPIC AND I'D LIKE YOU TO TALK ABOUT IT FOR ONE TO TWO MINUTES


RuntimeError: Calculated padded input size per channel: (0). Kernel size: (10). Kernel size can't be greater than actual input size